In [1]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
import pvlib
from pvlib.location import Location
import math
from math import cos
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 16, 6  # this sets the default image size for this session
from sklearn.metrics import r2_score
import scipy.stats

In [2]:
#BSRN QC
data_path ='DAA_2017_KD.csv'
df= pd.read_csv(data_path, index_col=[0], parse_dates=[0])
#df.index = df.index.tz_localize('Africa/Johannesburg')  # Make the index timezone aware
original_entries = df.shape[0]
df.head(2)  # Print the first two lines of the DataFrame

,zenith,k_d,kd_erbs,kd_erbsD,kd_ohm,kd_louche,kd_boland,kd_disc,kd_dirint,kd_dirindex,kd_Engerer2,kd_Engerer3,kd_Engerer4,kd_Yang5,kd_Yang4,kd_JKT,kd_Mabasa,k_de
DATE,,,,,,,,,,,,,,,,,,
2017-01-01 16:59:00+00:00,65.801731,0.999476,0.925253,0.925856,0.918115,0.868621,0.886417,0.899224,0.974559,0.974853,0.972142,0.940895,0.971137,0.952725,0.981976,0.976354,0.930707,0.0
2017-01-01 17:00:00+00:00,65.690040,0.998482,0.915752,0.916362,0.915650,0.858336,0.878193,0.887019,0.971478,0.971800,0.968582,0.933157,0.967336,0.949055,0.967715,0.968709,0.924315,0.0


In [3]:
df = df.replace(np.inf, np.nan).replace(-np.inf, np.nan).dropna()

In [4]:
df.columns

Index(['zenith', 'k_d', 'kd_erbs', 'kd_erbsD', 'kd_ohm', 'kd_louche',
       'kd_boland', 'kd_disc', 'kd_dirint', 'kd_dirindex', 'kd_Engerer2',
       'kd_Engerer3', 'kd_Engerer4', 'kd_Yang5', 'kd_Yang4', 'kd_JKT',
       'kd_Mabasa', 'k_de'],
      dtype='object')

In [5]:
df.describe()

,zenith,k_d,kd_erbs,kd_erbsD,kd_ohm,kd_louche,kd_boland,kd_disc,kd_dirint,kd_dirindex,kd_Engerer2,kd_Engerer3,kd_Engerer4,kd_Yang5,kd_Yang4,kd_JKT,kd_Mabasa,k_de
count,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000
mean,52.781324,0.267980,0.335362,0.333380,0.342540,0.296648,0.337853,0.300834,0.318790,0.295545,0.352295,0.278255,0.356016,0.322947,0.355443,0.377198,0.304470,0.452337
std,18.769075,1.065730,0.239311,0.237719,0.228883,0.239085,0.240359,0.221338,0.240581,0.252894,0.257599,0.239322,0.250111,0.260736,0.276265,0.277897,0.238163,28.787273
min,11.604543,-229.000000,0.164578,-0.000000,0.177000,0.105962,0.035204,0.125172,0.064271,-0.096259,0.102667,0.109063,0.141723,0.080803,0.087891,0.077003,0.031008,0.000000
25%,38.833994,0.113494,0.165000,0.165000,0.177000,0.116948,0.155216,0.156120,0.151957,0.112619,0.160994,0.120847,0.176222,0.135915,0.140602,0.147385,0.135352,0.000000
50%,55.172574,0.181351,0.202000,0.201772,0.212901,0.176270,0.238790,0.196633,0.196437,0.183264,0.235401,0.157062,0.233455,0.197761,0.226222,0.260967,0.201451,0.000000
75%,67.475066,0.366633,0.441146,0.437218,0.455118,0.415809,0.473547,0.359482,0.436549,0.415417,0.468733,0.330007,0.458207,0.427658,0.511880,0.568999,0.389181,0.000000
max,84.999660,0.999785,1.000000,0.995326,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.999504,1.000000,1.000000,1.000000,0.992396,1.000000,7605.460000


In [6]:
k_d = df['k_d'] #y_test #reference #observation
Engerer2 = df['kd_Engerer2'] #predicted
Engerer3 = df['kd_Engerer3'] #predicted
Engerer4 = df['kd_Engerer4'] #predicted
Yang4 = df['kd_Yang4'] #predicted
Yang5 = df['kd_Yang5'] #predicted
#Mabasa = df['kd_Mabasa'] #predicted
JKT = df['kd_JKT'] #predicted
dirindex = df['kd_dirindex'] #predicted
dirint = df['kd_dirint'] #predicted
disc = df['kd_disc'] #predicted
erbs = df['kd_erbs'] #predicted
erbsD = df['kd_erbsD'] #predicted
ohm = df['kd_ohm'] #predicted
boland = df['kd_boland'] #predicted
louche = df['kd_louche'] #predicted
Mabasa = df['kd_Mabasa'] #predicted

In [7]:
k_d_mean = df['k_d'].mean()
Engerer2_mean = df['kd_Engerer2'].mean()#predicted
Engerer3_mean = df['kd_Engerer3'].mean() #predicted
Engerer4_mean = df['kd_Engerer4'].mean() #predicted
Yang4_mean = df['kd_Yang4'].mean()#predicted
Yang5_mean = df['kd_Yang5'].mean()#predicted
Mabasa_mean = df['kd_Mabasa'].mean()#predicted
JKT_mean = df['kd_JKT'].mean()#predicted
dirindex_mean = df['kd_dirindex'].mean()#predicted
dirint_mean = df['kd_dirint'].mean()#predicted
disc_mean = df['kd_disc'].mean()#predicted
erbs_mean = df['kd_erbs'].mean()#predicted
erbsD_mean = df['kd_erbsD'].mean()#predicted
ohm_mean = df['kd_ohm'].mean()#predicted
boland_mean = df['kd_boland'].mean()#predicted
louche_mean = df['kd_louche'].mean()#predicted

In [8]:
#The correlation coefficient r is directly related to the coefficient of determination r2 in the obvious way.
#If r2 is represented in decimal form,If r2 is represented in decimal form, e.g. 0.39 or 0.87, 
#then all we have to do to obtain r is to take the square root of r2:
#Estimated slope and the correlation coefficient r always share the same sign. 
#Furthermore, because r2 is always a number between 0 and 1, the correlation coefficient r is always a number between -1 and 1.

In [9]:
MBE_Engerer2 = np.mean(Engerer2 - k_d)
rMBE_Engerer2 = np.mean(Engerer2 - k_d)/np.mean(k_d)*100
RMSE_Engerer2 = np.sqrt(np.mean((Engerer2 - k_d)**2))
rRMSE_Engerer2 =np.sqrt(np.mean((Engerer2 - k_d)**2)) / np.mean(k_d)*100
MAE_Engerer2 = np.mean(abs(Engerer2 - k_d))
rMAE_Engerer2 = np.mean(abs(Engerer2 - k_d)) / np.mean(k_d)*100
r_Engerer2 , p = scipy.stats.pearsonr(k_d, Engerer2) #Pearson correlation coefficient r (-1,1)
r2_Engerer2 = r_Engerer2 **2 #Coefficient of determination r2 (0,1)
summary_Engerer2 = pd.DataFrame(index = ['k_d_mean','Engerer2_mean','MBE_Engerer2','rMBE_Engerer2','RMSE_Engerer2', 'rRMSE_Engerer2', 'MAE_Engerer2', 'rMAE_Engerer2', 'r2_Engerer2']) 
summary_Engerer2['Linear Regression, all variables'] = [k_d_mean,Engerer2_mean,MBE_Engerer2,rMBE_Engerer2,RMSE_Engerer2,rRMSE_Engerer2,MAE_Engerer2, rMAE_Engerer2, r2_Engerer2]
#summary_Engerer2.to_csv('UPN_ALLSKY_DNI_Engerer2.csv')
summary_Engerer2

,"Linear Regression, all variables"
k_d_mean,0.267980
Engerer2_mean,0.352295
MBE_Engerer2,0.084315
rMBE_Engerer2,31.463334
RMSE_Engerer2,1.061363
rRMSE_Engerer2,396.060922
MAE_Engerer2,0.127698
rMAE_Engerer2,47.652135
r2_Engerer2,0.022715


In [10]:
MBE_Mabasa = np.mean(Mabasa - k_d)
rMBE_Mabasa = np.mean(Mabasa - k_d)/np.mean(k_d)*100
RMSE_Mabasa = np.sqrt(np.mean((Mabasa - k_d)**2))
rRMSE_Mabasa =np.sqrt(np.mean((Mabasa - k_d)**2)) / np.mean(k_d)*100
MAE_Mabasa = np.mean(abs(Mabasa - k_d))
rMAE_Mabasa = np.mean(abs(Mabasa - k_d)) / np.mean(k_d)*100
r_Mabasa , p = scipy.stats.pearsonr(k_d, Mabasa) #Pearson correlation coefficient r (-1,1)
r2_Mabasa = r_Mabasa **2 #Coefficient of determination r2 (0,1)
summary_Mabasa = pd.DataFrame(index = ['k_d_mean','Mabasa_mean','MBE_Mabasa','rMBE_Mabasa','RMSE_Mabasa', 'rRMSE_Mabasa', 'MAE_Mabasa', 'rMAE_Mabasa', 'r2_Mabasa']) 
summary_Mabasa['Linear Regression, all variables'] = [k_d_mean,Mabasa_mean,MBE_Mabasa,rMBE_Mabasa,RMSE_Mabasa,rRMSE_Mabasa,MAE_Mabasa, rMAE_Mabasa, r2_Mabasa]
#summary_Mabasa.to_csv('UPN_ALLSKY_DNI_Mabasa.csv')
summary_Mabasa

,"Linear Regression, all variables"
k_d_mean,0.267980
Mabasa_mean,0.304470
MBE_Mabasa,0.036490
rMBE_Mabasa,13.616693
RMSE_Mabasa,1.058315
rRMSE_Mabasa,394.923670
MAE_Mabasa,0.106475
rMAE_Mabasa,39.732616
r2_Mabasa,0.021134


In [11]:
#def root_mean_square_deviation(Mabasa, kd):
#Mabasa = np.array(Mabasa)
#kd = np.array(kd)
#rmse = np.sqrt(np.mean((Mabasa - kd) ** 2))
#rmse
#def mean_bias_deviation(Mabasa, kd):
#Mabasa = np.array(Mabasa)
#kd = np.array(kd)
#mbd = np.mean(Mabasa - kd)
#mbd
#def r_squared(Mabasa, kd):
#Mabasa = np.array(Mabasa)
#kd = np.array(kd)
#ss_res = np.sum((kd - Mabasa) ** 2)
#ss_tot = np.sum((kd - np.mean(kd)) ** 2)
#r2 = 1 - (ss_res / ss_tot)
#r2

In [12]:
MBE_Engerer3 = np.mean(Engerer3 - k_d)
rMBE_Engerer3 = np.mean(Engerer3 - k_d)/np.mean(k_d)*100
RMSE_Engerer3 = np.sqrt(np.mean((Engerer3 - k_d)**2))
rRMSE_Engerer3 =np.sqrt(np.mean((Engerer3 - k_d)**2)) / np.mean(k_d)*100
MAE_Engerer3 = np.mean(abs(Engerer3 - k_d))
rMAE_Engerer3 = np.mean(abs(Engerer3 - k_d)) / np.mean(k_d)*100
r_Engerer3 , p = scipy.stats.pearsonr(k_d, Engerer3) #Pearson correlation coefficient r (-1,1)
r2_Engerer3 = r_Engerer3 **2 #Coefficient of determination r2 (0,1)
summary_Engerer3 = pd.DataFrame(index = ['k_d_mean','Engerer3_mean','MBE_Engerer3','rMBE_Engerer3','RMSE_Engerer3','rRMSE_Engerer3', 'MAE_Engerer3','rMAE_Engerer3','r2_Engerer3']) 
summary_Engerer3['Linear Regression, all variables'] = [k_d_mean,Engerer3_mean,MBE_Engerer3,rMBE_Engerer3,RMSE_Engerer3,rRMSE_Engerer3,MAE_Engerer3, rMAE_Engerer3, r2_Engerer3]
#summary_Engerer2.to_csv('UPN_ALLSKY_DNI_Engerer2.csv')
summary_Engerer3

,"Linear Regression, all variables"
k_d_mean,0.267980
Engerer3_mean,0.278255
MBE_Engerer3,0.010275
rMBE_Engerer3,3.834394
RMSE_Engerer3,1.062890
rRMSE_Engerer3,396.630630
MAE_Engerer3,0.127116
rMAE_Engerer3,47.434987
r2_Engerer3,0.015458


In [13]:
MBE_Engerer4 = np.mean(Engerer4 - k_d)
rMBE_Engerer4 = np.mean(Engerer4 - k_d)/np.mean(k_d)*100
RMSE_Engerer4 = np.sqrt(np.mean((Engerer4 - k_d)**2))
rRMSE_Engerer4 =np.sqrt(np.mean((Engerer4 - k_d)**2)) / np.mean(k_d)*100
MAE_Engerer4 = np.mean(abs(Engerer4 - k_d))
rMAE_Engerer4 = np.mean(abs(Engerer4 - k_d)) / np.mean(k_d)*100
r_Engerer4 , p = scipy.stats.pearsonr(k_d, Engerer4) #Pearson correlation coefficient r (-1,1)
r2_Engerer4 = r_Engerer4 **2 #Coefficient of determination r2 (0,1)
summary_Engerer4 = pd.DataFrame(index = ['k_d_mean','Engerer4_mean','MBE_Engerer4','rMBE_Engerer4','RMSE_Engerer4', 'rRMSE_Engerer4', 'MAE_Engerer4', 'rMAE_Engerer4', 'r2_Engerer4']) 
summary_Engerer4['Linear Regression, all variables'] = [k_d_mean,Engerer4_mean,MBE_Engerer4,rMBE_Engerer4,RMSE_Engerer4,rRMSE_Engerer4,MAE_Engerer4, rMAE_Engerer4, r2_Engerer4]
#summary_Engerer2.to_csv('UPN_ALLSKY_DNI_Engerer2.csv')
summary_Engerer4

,"Linear Regression, all variables"
k_d_mean,0.267980
Engerer4_mean,0.356016
MBE_Engerer4,0.088037
rMBE_Engerer4,32.852015
RMSE_Engerer4,1.062438
rRMSE_Engerer4,396.462100
MAE_Engerer4,0.140535
rMAE_Engerer4,52.442318
r2_Engerer4,0.021029


In [14]:
MBE_Yang4 = np.mean(Yang4 - k_d)
rMBE_Yang4 = np.mean(Yang4 - k_d)/np.mean(k_d)*100
RMSE_Yang4 = np.sqrt(np.mean((Yang4 - k_d)**2))
rRMSE_Yang4 =np.sqrt(np.mean((Yang4 - k_d)**2)) / np.mean(k_d)*100
MAE_Yang4 = np.mean(abs(Yang4 - k_d))
rMAE_Yang4 = np.mean(abs(Yang4 - k_d)) / np.mean(k_d)*100
r_Yang4 , p = scipy.stats.pearsonr(k_d, Yang4) #Pearson correlation coefficient r (-1,1)
r2_Yang4 = r_Yang4 **2 #Coefficient of determination r2 (0,1)
summary_Yang4 = pd.DataFrame(index = ['k_d_mean','Yang4_mean','MBE_Yang4','rMBE_Yang4','RMSE_Yang4', 'rRMSE_Yang4', 'MAE_Yang4', 'rMAE_Yang4', 'r2_Yang4']) 
summary_Yang4 ['Linear Regression, all variables'] = [k_d_mean,Yang4_mean,MBE_Yang4,rMBE_Yang4,RMSE_Yang4,rRMSE_Yang4,MAE_Yang4, rMAE_Yang4, r2_Yang4]
#summary_Engerer2.to_csv('UPN_ALLSKY_DNI_Engerer2.csv')
summary_Yang4

,"Linear Regression, all variables"
k_d_mean,0.267980
Yang4_mean,0.355443
MBE_Yang4,0.087463
rMBE_Yang4,32.638051
RMSE_Yang4,1.061885
rRMSE_Yang4,396.255643
MAE_Yang4,0.120560
rMAE_Yang4,44.988605
r2_Yang4,0.024489


In [15]:
MBE_Yang5 = np.mean(Yang5 - k_d)
rMBE_Yang5 = np.mean(Yang5 - k_d)/np.mean(k_d)*100
RMSE_Yang5 = np.sqrt(np.mean((Yang5 - k_d)**2))
rRMSE_Yang5 =np.sqrt(np.mean((Yang5 - k_d)**2)) / np.mean(k_d)*100
MAE_Yang5 = np.mean(abs(Yang5 - k_d))
rMAE_Yang5 = np.mean(abs(Yang5 - k_d)) / np.mean(k_d)*100
r_Yang5 , p = scipy.stats.pearsonr(k_d, Yang5) #Pearson correlation coefficient r (-1,1)
r2_Yang5 = r_Yang5 **2 #Coefficient of determination r2 (0,1)
summary_Yang5 = pd.DataFrame(index = ['k_d_mean','Yang5_mean','MBE_Yang5','rMBE_Yang5','RMSE_Yang5', 'rRMSE_Yang5', 'MAE_Yang5', 'rMAE_Yang5', 'r2_Yang5']) 
summary_Yang5 ['Linear Regression, all variables'] = [k_d_mean,Yang5_mean,MBE_Yang5,rMBE_Yang5,RMSE_Yang5,rRMSE_Yang5,MAE_Yang5, rMAE_Yang5, r2_Yang5]
#summary_Engerer2.to_csv('UPN_ALLSKY_DNI_Engerer2.csv')
summary_Yang5

,"Linear Regression, all variables"
k_d_mean,0.267980
Yang5_mean,0.322947
MBE_Yang5,0.054967
rMBE_Yang5,20.511628
RMSE_Yang5,1.059955
rRMSE_Yang5,395.535580
MAE_Yang5,0.113661
rMAE_Yang5,42.413878
r2_Yang5,0.022453


In [16]:
MBE_JKT = np.mean(JKT - k_d)
rMBE_JKT = np.mean(JKT - k_d)/np.mean(k_d)*100
RMSE_JKT = np.sqrt(np.mean((JKT - k_d)**2))
rRMSE_JKT =np.sqrt(np.mean((JKT - k_d)**2)) / np.mean(k_d)*100
MAE_JKT = np.mean(abs(JKT - k_d))
rMAE_JKT = np.mean(abs(JKT - k_d)) / np.mean(k_d)*100
r_JKT, p = scipy.stats.pearsonr(k_d, JKT) #Pearson correlation coefficient r (-1,1)
r2_JKT = r_JKT **2 #Coefficient of determination r2 (0,1)
summary_JKT = pd.DataFrame(index = ['k_d_mean','JKT_mean','MBE_JKT','rMBE_JKT','RMSE_JKT','rRMSE_JKT', 'MAE_JKT', 'rMAE_JKT','r2_JKT']) 
summary_JKT  ['Linear Regression, all variables'] = [k_d_mean,JKT_mean,MBE_JKT,rMBE_JKT,RMSE_JKT,rRMSE_JKT,MAE_JKT, rMAE_JKT, r2_JKT]
#summary_Engerer2.to_csv('UPN_ALLSKY_DNI_Engerer2.csv')
summary_JKT

,"Linear Regression, all variables"
k_d_mean,0.267980
JKT_mean,0.377198
MBE_JKT,0.109218
rMBE_JKT,40.756218
RMSE_JKT,1.065523
rRMSE_JKT,397.613255
MAE_JKT,0.143786
rMAE_JKT,53.655506
r2_JKT,0.022878


In [17]:
MBE_erbs =np.mean(erbs - k_d)
rMBE_erbs=np.mean(erbs - k_d)/np.mean(k_d)*100
RMSE_erbs = np.sqrt(np.mean((erbs - k_d)**2))
rRMSE_erbs =np.sqrt(np.mean((erbs - k_d)**2)) / np.mean(k_d)*100
MAE_erbs = np.mean(abs(erbs - k_d))
rMAE_erbs = np.mean(abs(erbs - k_d)) / np.mean(k_d)*100
r_erbs, p = scipy.stats.pearsonr(k_d, erbs) #Pearson correlation coefficient
r2_erbs=r_erbs **2
summary_erbs = pd.DataFrame(index = ['k_d_mean','erbs_mean','MBE_erbs','rMBE_erbs','RMSE_erbs', 'rRMSE_erbs', 'MAE_erbs', 'rMAE_erbs', 'r2_erbs']) 
summary_erbs['Linear Regression, all variables'] = [k_d_mean, erbs_mean,MBE_erbs,rMBE_erbs,RMSE_erbs, rRMSE_erbs, MAE_erbs, rMAE_erbs, r2_erbs]
#summary_louche.to_csv('UPN_ALLSKY_DNI_LOUCHE.csv')
summary_erbs

,"Linear Regression, all variables"
k_d_mean,0.267980
erbs_mean,0.335362
MBE_erbs,0.067382
rMBE_erbs,25.144498
RMSE_erbs,1.062032
rRMSE_erbs,396.310714
MAE_erbs,0.137241
rMAE_erbs,51.213308
r2_erbs,0.018658


In [18]:
MBE_erbsD =np.mean(erbsD - k_d)
rMBE_erbsD=np.mean(erbsD - k_d)/np.mean(k_d)*100
RMSE_erbsD = np.sqrt(np.mean((erbsD - k_d)**2))
rRMSE_erbsD =np.sqrt(np.mean((erbsD - k_d)**2)) / np.mean(k_d)*100
MAE_erbsD = np.mean(abs(erbsD - k_d))
rMAE_erbsD = np.mean(abs(erbsD - k_d)) / np.mean(k_d)*100
r_erbsD, p = scipy.stats.pearsonr(k_d, erbsD) #Pearson correlation coefficient
r2_erbsD=r_erbsD **2
summary_erbsD = pd.DataFrame(index = ['k_d_mean','erbsD_mean','MBE_erbsD','rMBE_erbsD','RMSE_erbsD', 'rRMSE_erbsD', 'MAE_erbsD', 'rMAE_erbsD', 'r2_erbsD']) 
summary_erbsD ['Linear Regression, all variables'] = [k_d_mean, erbsD_mean,MBE_erbsD,rMBE_erbsD,RMSE_erbsD, rRMSE_erbsD, MAE_erbsD, rMAE_erbsD, r2_erbsD]
#summary_louche.to_csv('UPN_ALLSKY_DNI_LOUCHE.csv')
summary_erbsD

,"Linear Regression, all variables"
k_d_mean,0.267980
erbsD_mean,0.333380
MBE_erbsD,0.065400
rMBE_erbsD,24.404933
RMSE_erbsD,1.045122
rRMSE_erbsD,390.000286
MAE_erbsD,0.135015
rMAE_erbsD,50.382693
r2_erbsD,0.042360


In [19]:
MBE_boland =np.mean(boland - k_d)
rMBE_boland=np.mean(boland - k_d)/np.mean(k_d)*100
RMSE_boland = np.sqrt(np.mean((boland - k_d)**2))
rRMSE_boland =np.sqrt(np.mean((boland - k_d)**2)) / np.mean(k_d)*100
MAE_boland = np.mean(abs(boland - k_d))
rMAE_boland = np.mean(abs(boland - k_d)) / np.mean(k_d)*100
r_boland , p = scipy.stats.pearsonr(k_d, boland) #Pearson correlation coefficient
r2_boland = r_boland **2
summary_boland = pd.DataFrame(index = ['k_d_mean','boland_mean','MBE_boland','rMBE_boland','RMSE_boland', 'rRMSE_boland', 'MAE_boland', 'rMAE_boland', 'r2_boland']) 
summary_boland ['Linear Regression, all variables'] = [k_d_mean, boland_mean,MBE_boland,rMBE_boland,RMSE_boland, rRMSE_boland, MAE_boland, rMAE_boland, r2_boland]
#summary_louche.to_csv('UPN_ALLSKY_DNI_LOUCHE.csv')
summary_boland

,"Linear Regression, all variables"
k_d_mean,0.267980
boland_mean,0.337853
MBE_boland,0.069873
rMBE_boland,26.073924
RMSE_boland,1.063199
rRMSE_boland,396.745965
MAE_boland,0.145215
rMAE_boland,54.188742
r2_boland,0.017638


In [20]:
MBE_louche =np.mean(louche - k_d)
rMBE_louche=np.mean(louche - k_d)/np.mean(k_d)*100
RMSE_louche = np.sqrt(np.mean((louche - k_d)**2))
rRMSE_louche =np.sqrt(np.mean((louche - k_d)**2)) / np.mean(k_d)*100
MAE_louche = np.mean(abs(louche - k_d))
rMAE_louche = np.mean(abs(louche - k_d)) / np.mean(k_d)*100
r_louche, p = scipy.stats.pearsonr(k_d, louche) #Pearson correlation coefficient
r2_louche=r_louche**2
summary_louche = pd.DataFrame(index = ['k_d_mean','louche_mean','MBE_louche','rMBE_louche','RMSE_louche', 'rRMSE_louche', 'MAE_louche', 'rMAE_louche', 'r2_louche']) 
summary_louche['Linear Regression, all variables'] = [k_d_mean, louche_mean,MBE_louche, rMBE_louche, RMSE_louche, rRMSE_louche, MAE_louche, rMAE_louche, r2_louche]
#summary_louche.to_csv('UPN_ALLSKY_DNI_LOUCHE.csv')
summary_louche

,"Linear Regression, all variables"
k_d_mean,0.267980
louche_mean,0.296648
MBE_louche,0.028668
rMBE_louche,10.697760
RMSE_louche,1.060882
rRMSE_louche,395.881617
MAE_louche,0.120666
rMAE_louche,45.027891
r2_louche,0.017957


In [21]:
MBE_ohm =np.mean(ohm - k_d)
rMBE_ohm =np.mean(ohm - k_d)/np.mean(k_d)*100
RMSE_ohm = np.sqrt(np.mean((ohm - k_d)**2))
rRMSE_ohm =np.sqrt(np.mean((ohm - k_d)**2)) / np.mean(k_d)*100
MAE_ohm = np.mean(abs(ohm - k_d))
rMAE_ohm = np.mean(abs(ohm - k_d)) / np.mean(k_d)*100
r_ohm, p = scipy.stats.pearsonr(k_d, ohm) #Pearson correlation coefficient
r2_ohm =r_ohm **2
summary_ohm = pd.DataFrame(index = ['k_d_mean','ohm_mean','MBE_ohm','rMBE_louche','RMSE_ohm', 'rRMSE_ohm', 'MAE_ohm', 'rMAE_ohm', 'r2_ohm']) 
summary_ohm ['Linear Regression, all variables'] = [k_d_mean, ohm_mean,MBE_ohm,rMBE_ohm, RMSE_ohm, rRMSE_ohm, MAE_ohm, rMAE_ohm, r2_ohm]
#summary_louche.to_csv('UPN_ALLSKY_k_d_LOUCHE.csv')
summary_ohm

,"Linear Regression, all variables"
k_d_mean,0.267980
ohm_mean,0.342540
MBE_ohm,0.074561
rMBE_louche,27.823230
RMSE_ohm,1.062041
rRMSE_ohm,396.313852
MAE_ohm,0.141804
rMAE_ohm,52.915772
r2_ohm,0.018187


In [22]:
MBE_dirindex = np.mean(dirindex - k_d)
rMBE_dirindex = np.mean(dirindex - k_d)/np.mean(k_d)*100
RMSE_dirindex = np.sqrt(np.mean((dirindex - k_d)**2))
rRMSE_dirindex =np.sqrt(np.mean((dirindex - k_d)**2)) / np.mean(k_d)*100
MAE_dirindex = np.mean(abs(dirindex - k_d))
rMAE_dirindex = np.mean(abs(dirindex - k_d)) / np.mean(k_d)*100
r_dirindex, p = scipy.stats.pearsonr(k_d, dirindex) #Pearson correlation coefficient r (-1,1)
r2_dirindex = r_dirindex **2 #Coefficient of determination r2 (0,1)
summary_dirindex = pd.DataFrame(index = ['k_d_mean','dirindex_mean','MBE_dirindex','rMBE_dirindex','RMSE_dirindex','rRMSE_dirindex', 'MAE_dirindex', 'rMAE_dirindex','r2_dirindex']) 
summary_dirindex  ['Linear Regression, all variables'] = [k_d_mean,dirindex_mean,MBE_dirindex,rMBE_dirindex,RMSE_dirindex,rRMSE_dirindex,MAE_dirindex, rMAE_dirindex, r2_dirindex]
#summary_Engerer2.to_csv('UPN_ALLSKY_k_d_Engerer2.csv')
summary_dirindex

,"Linear Regression, all variables"
k_d_mean,0.267980
dirindex_mean,0.295545
MBE_dirindex,0.027566
rMBE_dirindex,10.286436
RMSE_dirindex,1.057378
rRMSE_dirindex,394.574016
MAE_dirindex,0.097886
rMAE_dirindex,36.527516
r2_dirindex,0.023392


In [23]:
MBE_dirint = np.mean(dirint - k_d)
rMBE_dirint = np.mean(dirint - k_d)/np.mean(k_d)*100
RMSE_dirint = np.sqrt(np.mean((dirint - k_d)**2))
rRMSE_dirint =np.sqrt(np.mean((dirint - k_d)**2)) / np.mean(k_d)*100
MAE_dirint = np.mean(abs(dirint - k_d))
rMAE_dirint = np.mean(abs(dirint - k_d)) / np.mean(k_d)*100
r_dirint, p = scipy.stats.pearsonr(k_d, dirint) #Pearson correlation coefficient r (-1,1)
r2_dirint = r_dirint **2 #Coefficient of determination r2 (0,1)
summary_dirint = pd.DataFrame(index = ['k_d_mean','dirint_mean','MBE_dirint','rMBE_dirint','RMSE_dirint','rRMSE_dirint', 'MAE_dirint', 'rMAE_dirint','r2_dirint']) 
summary_dirint  ['Linear Regression, all variables'] = [k_d_mean,dirint_mean,MBE_dirint,rMBE_dirint,RMSE_dirint,rRMSE_dirint,MAE_dirint, rMAE_dirint, r2_dirint]
#summary_Engerer2.to_csv('UPN_ALLSKY_k_d_Engerer2.csv')
summary_dirint

,"Linear Regression, all variables"
k_d_mean,0.267980
dirint_mean,0.318790
MBE_dirint,0.050810
rMBE_dirint,18.960411
RMSE_dirint,1.057656
rRMSE_dirint,394.677565
MAE_dirint,0.105738
rMAE_dirint,39.457374
r2_dirint,0.022901


In [24]:
MBE_disc = np.mean(disc - k_d)
rMBE_disc = np.mean(disc - k_d)/np.mean(k_d)*100
RMSE_disc = np.sqrt(np.mean((disc - k_d)**2))
rRMSE_disc =np.sqrt(np.mean((disc - k_d)**2)) / np.mean(k_d)*100
MAE_disc = np.mean(abs(disc - k_d))
rMAE_disc = np.mean(abs(disc - k_d)) / np.mean(k_d)*100
r_disc, p = scipy.stats.pearsonr(k_d, disc) #Pearson correlation coefficient r (-1,1)
r2_disc = r_disc **2 #Coefficient of determination r2 (0,1)
summary_disc = pd.DataFrame(index = ['k_d_mean','disc_mean','MBE_disc','rMBE_disc','RMSE_disc','rRMSE_disc', 'MAE_disc', 'rMAE_disc','r2_disc']) 
summary_disc  ['Linear Regression, all variables'] = [k_d_mean,disc_mean,MBE_disc,rMBE_disc,RMSE_disc,rRMSE_disc,MAE_disc, rMAE_dirint, r2_disc]
#summary_Engerer2.to_csv('UPN_ALLSKY_k_d_Engerer2.csv')
summary_disc

,"Linear Regression, all variables"
k_d_mean,0.267980
disc_mean,0.300834
MBE_disc,0.032854
rMBE_disc,12.260038
RMSE_disc,1.057726
rRMSE_disc,394.703706
MAE_disc,0.106365
rMAE_disc,39.457374
r2_disc,0.020206


In [25]:
DEAAR_RMSE=pd.DataFrame([RMSE_dirindex, RMSE_dirint, RMSE_disc, RMSE_JKT, RMSE_Mabasa, RMSE_Yang5, RMSE_Yang4, RMSE_Engerer4, RMSE_Engerer3, RMSE_Engerer2, RMSE_erbs, RMSE_erbsD, RMSE_louche, RMSE_boland,RMSE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_RMSE.to_csv('RMSE_UINGTON_ALLSKY_kd.csv')
DEAAR_RMSE

,0
dirindex,1.057378
dirint,1.057656
disc,1.057726
JKT,1.065523
Mabasa,1.058315
Yang5,1.059955
Yang4,1.061885
Engerer4,1.062438
Engerer3,1.062890
Engerer2,1.061363


In [26]:
DEAAR_rRMSE=pd.DataFrame([rRMSE_dirindex, rRMSE_dirint, rRMSE_disc,rRMSE_JKT, rRMSE_Mabasa,rRMSE_Yang5, rRMSE_Yang4, rRMSE_Engerer4, rRMSE_Engerer3, rRMSE_Engerer2, rRMSE_erbs, rRMSE_erbsD, rRMSE_louche, rRMSE_boland,rRMSE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_rRMSE.to_csv('rRMSE_UINGTON_ALLSKY_kd.csv')
DEAAR_rRMSE

,0
dirindex,394.574016
dirint,394.677565
disc,394.703706
JKT,397.613255
Mabasa,394.923670
Yang5,395.535580
Yang4,396.255643
Engerer4,396.462100
Engerer3,396.630630
Engerer2,396.060922


In [27]:
DEAAR_MBE=pd.DataFrame([MBE_dirindex, MBE_dirint, MBE_disc, MBE_JKT, MBE_Mabasa, MBE_Yang5, MBE_Yang4, MBE_Engerer4, MBE_Engerer3, MBE_Engerer2, MBE_erbs, MBE_erbsD,MBE_louche, MBE_boland,MBE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_MBE.to_csv('MBE_UINGTON_ALLSKY_kd.csv')
DEAAR_MBE

,0
dirindex,0.027566
dirint,0.050810
disc,0.032854
JKT,0.109218
Mabasa,0.036490
Yang5,0.054967
Yang4,0.087463
Engerer4,0.088037
Engerer3,0.010275
Engerer2,0.084315


In [28]:
DEAAR_rMBE=pd.DataFrame([rMBE_dirindex, rMBE_dirint, rMBE_disc, rMBE_JKT, rMBE_Mabasa, rMBE_Yang5, rMBE_Yang4, rMBE_Engerer4, rMBE_Engerer3, rMBE_Engerer2, rMBE_erbs, rMBE_erbsD,rMBE_louche, rMBE_boland,rMBE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_rMBE.to_csv('rMBE_UINGTON_ALLSKY_kd.csv')
DEAAR_rMBE

,0
dirindex,10.286436
dirint,18.960411
disc,12.260038
JKT,40.756218
Mabasa,13.616693
Yang5,20.511628
Yang4,32.638051
Engerer4,32.852015
Engerer3,3.834394
Engerer2,31.463334


In [29]:
DEAAR_MAE=pd.DataFrame([MAE_dirindex, MAE_dirint, MAE_disc, MAE_JKT, MAE_Mabasa, MAE_Yang5, MAE_Yang4, MAE_Engerer4, MAE_Engerer3, MAE_Engerer2, MAE_erbs, MAE_erbsD,MAE_louche, MAE_boland,MAE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_MAE.to_csv('MAE_UINGTON_ALLSKY_kd.csv')
DEAAR_MAE

,0
dirindex,0.097886
dirint,0.105738
disc,0.106365
JKT,0.143786
Mabasa,0.106475
Yang5,0.113661
Yang4,0.120560
Engerer4,0.140535
Engerer3,0.127116
Engerer2,0.127698


In [30]:
DEAAR_rMAE=pd.DataFrame([rMAE_dirindex, rMAE_dirint, rMAE_disc, rMAE_JKT, rMAE_Mabasa, rMAE_Yang5, rMAE_Yang4, rMAE_Engerer4, rMAE_Engerer3, rMAE_Engerer2, rMAE_erbs, rMAE_erbsD,rMAE_louche, rMAE_boland,rMAE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_rMAE.to_csv('rMAE_UINGTON_ALLSKY_kd.csv')
DEAAR_rMAE

,0
dirindex,36.527516
dirint,39.457374
disc,39.691481
JKT,53.655506
Mabasa,39.732616
Yang5,42.413878
Yang4,44.988605
Engerer4,52.442318
Engerer3,47.434987
Engerer2,47.652135


In [31]:
DEAAR_r2=pd.DataFrame([r2_dirindex, r2_dirint, r2_disc, r2_JKT, r2_Mabasa, r2_Yang5, r2_Yang4, r2_Engerer4, r2_Engerer3, r2_Engerer2, r2_erbs, r2_erbsD,r2_louche, r2_boland,r2_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_r2.to_csv('r2_UINGTON_ALLSKY_kd.csv')
DEAAR_r2

,0
dirindex,0.023392
dirint,0.022901
disc,0.020206
JKT,0.022878
Mabasa,0.021134
Yang5,0.022453
Yang4,0.024489
Engerer4,0.021029
Engerer3,0.015458
Engerer2,0.022715


In [32]:
DEAAR_RMSE.to_csv('RMSE_DEAAR_ALLSKY_kd.csv')
DEAAR_rRMSE.to_csv('rRMSE_DEAAR_ALLSKY_kd.csv')
DEAAR_MBE.to_csv('MBE_DEAAR_ALLSKY_kd.csv')
DEAAR_rMBE.to_csv('rMBE_DEAAR_ALLSKY_kd.csv')
DEAAR_MAE.to_csv('MAE_DEAAR_ALLSKY_kd.csv')
DEAAR_rMAE.to_csv('rMAE_DEAAR_ALLSKY_kd.csv')
DEAAR_r2.to_csv('r2_DEAAR_ALLSKY_kd.csv')